# PWG2. Play Atari in the gym



![](./pics/blue.png)

## 0. 使用前的一些建议

Atari 是老版游戏机上的那种小游戏。这里我们使用 gym 作为载体，让强化学习的 agent 通过直接读取游戏的画面作为输入，来玩小游戏，并获取游戏内置的高分奖励。

在开始之前，需要先提醒各位的是，Gym 的 atari 只在 Linux 和 MacOS 系统可以使用。另外最好有显卡加速训练。这里由于我自己的电脑没有 GPU 因此我选择在 Colab 上运行。我的 Colab 笔记本位于 [PWG2_Play_Atari](https://colab.research.google.com/drive/12OWnh6p0Yp5JZFsfH9Nojd4ymyTUIrtL?usp=sharing)

如果你使用的是 Windows 电脑，那么则需要使用特别的安装方法：[How to Install OpenAI Gym in a Windows Environment](https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30) 

我试了一下上面的方法，对于我 Win11 系统来说似乎并不有效，Win10 我并不了解。但我建议各位如果没有Linux 系统的话可以尝试使用 WLS2 即 (Windows subsystem for Linux) 具体的使用方法。具体操作看视频：
- 如果你是 Win10 系统 [Youtube | Developing on Windows with WSL2 (Subsystem for Linux), VS Code, Docker, and the Terminal](https://www.youtube.com/watch?v=A0eqZujVfYU&ab_channel=ScottHanselman)
- 如果你是 Win11 系统 [Youtube | Windows 11 runs Graphical Linux Apps out of the box with WSLg](https://www.youtube.com/watch?v=b1YBx1L8op4&t=404s&ab_channel=ScottHanselman)

其实操作非常简单，只需要在 Windows 自带的应用商店里先下载一个 Windows terminal:

![Windows terminal](./pics/terminal.png)

搜索 Ubuntu，然后下载你喜欢的 Ubuntu 版本。

![ubuntu](./pics/ubuntu.png)

然后，当你打开 Windows Terminal 的时候就可以看到一个 Ubuntu 的图标了：

![WSL](./pics/ubuntu_in_terminal.png)

我这里说的比较概括，具体的操作建议跟着视频教程做。另外如果你是 Win11 的系统，那么恭喜你，你的电脑可以无缝在 Windows 图形化应用和 Linux 图形化应用之间切换。

如果你更喜欢使用 Vscode 而不是终端的话，建议看这个 VScode 官网文档： [Visual Studio Code Remote - WSL](https://code.visualstudio.com/docs/remote/wsl)

![](./pics/green.png)

## 1. 准备工作

为了让 Atari 能够在 Colab 上正常运行，我们需要先让 gym[Atari] 获取Colab 的 ROM。具体操作如下：

> 该段代码来自此 [Github](https://github.com/openai/atari-py/issues/83) page

In [ ]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/ -All
! python -m atari_py.import_roms /content/ROM/

由于 gym 环境的不断更新，有些环境的名字已经没有了。也有可能我们的 gym 包没有下全，导致有些环境不存在。所以在我们调用一个环境之前可以使用 `gym.envs.registry.namespace()` 函数查一下自己有没有这个环境：

In [1]:
import gym
from gym import envs
print(True if envs.registry.namespace('Adventure-v0') else False)

True


假如没有的话，也许是版本改变了，我们可以使用下面的代码查看环境的正确名称：

In [44]:
names = list(envs.registry.all())
keyword = 'Breakout'
print([name for name in names if keyword in str(name)])


[EnvSpec(Breakout-v0), EnvSpec(BreakoutDeterministic-v0), EnvSpec(BreakoutNoFrameskip-v0), EnvSpec(Breakout-v4), EnvSpec(BreakoutDeterministic-v4), EnvSpec(BreakoutNoFrameskip-v4), EnvSpec(Breakout-ram-v0), EnvSpec(Breakout-ramDeterministic-v0), EnvSpec(Breakout-ramNoFrameskip-v0), EnvSpec(Breakout-ram-v4), EnvSpec(Breakout-ramDeterministic-v4), EnvSpec(Breakout-ramNoFrameskip-v4)]


然后我们可以查看一个环境的基本信息。

In [3]:
def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

比如，我们来查看一下 Breakout 游戏：

In [5]:
query_environment('Breakout-v0')

Action Space: Discrete(4)
Observation Space: Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255

In [9]:
# create the Breakout environment
env = gym.make('Breakout-v0',render_mode='rgb_array')
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break
            
env.close()